In [71]:
import cv2
import numpy as np

In [72]:
IMG_DIR = '00-002/00/image_0/'
IMG_NAME_1 = '000000.png'
IMG_NAME_2 = '000001.png'

In [73]:
img_1 = cv2.imread(IMG_DIR + IMG_NAME_1)
img_2 = cv2.imread(IMG_DIR + IMG_NAME_2)
cv2.imshow('image1',img_1)
cv2.imshow('image2',img_2)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [74]:
sift = cv2.xfeatures2d.SIFT_create()
kp1, des1 = sift.detectAndCompute(img_1,None)
kp2, des2 = sift.detectAndCompute(img_2,None)
SIFT_1 = cv2.drawKeypoints(img_1, kp1, None, flags = cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
SIFT_2 = cv2.drawKeypoints(img_2, kp2, None, flags = cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
cv2.imshow('image1',SIFT_1)
cv2.imshow('image2',SIFT_2)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [75]:
FLANN_INDEX_KDTREE = 0
index_params = dict(algorithm = FLANN_INDEX_KDTREE, trees = 5)
search_params = dict(checks=50)
flann = cv2.FlannBasedMatcher(index_params,search_params)
matches = flann.knnMatch(des1,des2,k=2)
good = []
pts1 = []
pts2 = []
for i,(m,n) in enumerate(matches):
    if m.distance < 0.8*n.distance:
        good.append(m)
        pts2.append(kp2[m.trainIdx].pt)
        pts1.append(kp1[m.queryIdx].pt)

In [76]:
####Find EssentialMatrix
pts1_E = np.asarray(pts1)
pts2_E = np.asarray(pts2)
E, mask = cv2.findEssentialMat(pts1_E, pts2_E)
pts1_E = pts1_E[mask.ravel()==1]
pts2_E = pts2_E[mask.ravel()==1]
points, R, t, mask = cv2.recoverPose(E, pts1_E, pts2_E)

In [77]:
###Find Fundamental Matrix
pts1_F = np.int32(pts1)
pts2_F = np.int32(pts2)
F, mask = cv2.findFundamentalMat(pts1_F,pts2_F,cv2.FM_LMEDS)
pts1_F = pts1_F[mask.ravel()==1]
pts2_F = pts2_F[mask.ravel()==1]